In [1]:
!pip install torch torchvision

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 5
batch_size = 64



In [4]:
# Download and prepare MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transform_training = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomPerspective(distortion_scale=0.3, p=0.2),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform_training, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


100%|██████████| 9912422/9912422 [00:00<00:00, 95125866.57it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 27745234.50it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 26729632.24it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 22952444.30it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [6]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [7]:
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

print('Training complete!')

Epoch [1/5], Step [100/938], Loss: 0.2156
Epoch [1/5], Step [200/938], Loss: 0.1319
Epoch [1/5], Step [300/938], Loss: 0.1616
Epoch [1/5], Step [400/938], Loss: 0.1971
Epoch [1/5], Step [500/938], Loss: 0.0364
Epoch [1/5], Step [600/938], Loss: 0.0127
Epoch [1/5], Step [700/938], Loss: 0.1250
Epoch [1/5], Step [800/938], Loss: 0.0709
Epoch [1/5], Step [900/938], Loss: 0.1139
Epoch [2/5], Step [100/938], Loss: 0.0095
Epoch [2/5], Step [200/938], Loss: 0.0632
Epoch [2/5], Step [300/938], Loss: 0.0145
Epoch [2/5], Step [400/938], Loss: 0.0714
Epoch [2/5], Step [500/938], Loss: 0.0627
Epoch [2/5], Step [600/938], Loss: 0.0197
Epoch [2/5], Step [700/938], Loss: 0.1637
Epoch [2/5], Step [800/938], Loss: 0.1006
Epoch [2/5], Step [900/938], Loss: 0.1399
Epoch [3/5], Step [100/938], Loss: 0.0565
Epoch [3/5], Step [200/938], Loss: 0.0323
Epoch [3/5], Step [300/938], Loss: 0.0111
Epoch [3/5], Step [400/938], Loss: 0.0173
Epoch [3/5], Step [500/938], Loss: 0.2012
Epoch [3/5], Step [600/938], Loss:

In [9]:

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy on the test set: {100 * accuracy:.2f}%')

Accuracy on the test set: 99.14%
